In [1]:
# !pip install xlsx2csv

In [32]:
import polars as pl
import pandas as pd
import numpy as np

# GROUP CONTROL CHART

In [3]:
df = pl.read_excel("tabel_3_1.xlsx")
df

pad1,pad2,pad3,pad4
i64,i64,i64,i64
933,937,938,935
897,898,915,913
840,900,900,930
900,905,902,900
879,852,873,871
…,…,…,…
900,905,925,925
910,910,915,910
900,905,900,910


In [4]:
# Helper Function 1
def buat_kelompok(df):
    n = df.shape[0]
    grup = []
    num_kontrol = 1

    for i in range(int(n/2)):
        for j in range(2):        
            grup.append(f"{num_kontrol+i}-{num_kontrol+i+1}")
            num_kontrol += j
    return grup
 
# Helper Function 2
def tabelKelompok(df, n_pad):
    grup = buat_kelompok(df)
    df = df.with_columns(pl.Series(grup).alias('grup'))
    
    n = df.shape[0]
    genap = [x-1 for x in range(1, n + 1) if x % 2 != 0]
    ganjil = [x-1 for x in range(1, n + 1) if x % 2 == 0]

    a = []
    for i in genap:
        for j in range(n_pad):
            a.append(df[i,j])

    b = []
    for i in ganjil:
        for j in range(n_pad):
            b.append(df[i,j])

    df_kelompok = df.select([pl.col("grup").repeat_by(2).explode()])
    df_kelompok = df_kelompok.with_columns([
        pl.Series(a).alias("a"),
        pl.Series(b).alias("b")
    ])
    df_kelompok = df_kelompok.with_columns(pl.mean_horizontal(pl.col("a", "b")).alias("mean"))
    df_kelompok = df_kelompok.with_columns(R=(pl.col("a")-pl.col("b")).abs())
    return df_kelompok

In [5]:
tabel_kelompok = tabelKelompok(df=df, n_pad=4)
tabel_kelompok

grup,a,b,mean,R
str,i64,i64,f64,i64
"""1-2""",933,897,915.0,36
"""1-2""",937,898,917.5,39
"""1-2""",938,915,926.5,23
"""1-2""",935,913,924.0,22
"""3-4""",840,900,870.0,60
…,…,…,…,…
"""33-34""",910,910,910.0,0
"""35-36""",900,940,920.0,40
"""35-36""",950,938,944.0,12


In [6]:
X_mean = (tabel_kelompok.select([
    pl.mean("mean")
]))[0,0]
R_mean = (tabel_kelompok.select([
    pl.mean("R")
]))[0,0]

print(f"X mean = {X_mean}\nR_mean = {R_mean}")

X mean = 910.7222222222222
R_mean = 17.416666666666668


In [7]:
LKA_x_mean = X_mean + 1.88*R_mean
LKB_x_mean = X_mean - 1.88*R_mean
LKA_r_mean = 3.27*R_mean
LKB_r_mean = 0*R_mean

print(f"LKA_x_mean = {LKA_x_mean}\nLKB_x_mean = {LKB_x_mean}\nLKA_r_mean = {LKA_r_mean}\nLKB_r_mean = {LKB_r_mean}")

LKA_x_mean = 943.4655555555555
LKB_x_mean = 877.9788888888888
LKA_r_mean = 56.95250000000001
LKB_r_mean = 0.0


# Control Chart dengan n sampel tak sama

In [8]:
df = pl.read_excel("tabel_3_2.xlsx")
df = df.with_columns(
    [
        (pl.col("n")*pl.col("x_mean")).alias("nx"),
        (pl.col("n")*pl.col("s")**2).alias("ns^2")
    ]
)
df

mesin,n,x_mean,s,nx,ns^2
i64,i64,i64,i64,i64,f64
1,132,465,465,61380,2.85417e7
2,126,531,531,66906,3.5527086e7
3,128,561,561,71808,4.0284288e7
4,126,480,480,60480,2.90304e7
5,130,542,542,70460,3.818932e7
6,129,557,557,71853,4.0022121e7
7,120,604,604,72480,4.377792e7
8,117,573,573,67041,3.8414493e7
9,131,617,617,80827,4.9870259e7


In [9]:
x_mean_1 = (df.select([pl.mean("x_mean")]))[0,0]
x_mean_2 = (df.select([pl.sum("nx")/pl.sum("n")]))[0,0]

s_mean_1 = (df.select([pl.mean("s")]))[0,0]
s_mean_2 = (df.select([(pl.sum("ns^2")/pl.sum("n")).sqrt()]))[0,0]

print(f"x_mean_1 = {x_mean_1}\nx_mean_2 = {x_mean_2}\ns_mean_1 = {s_mean_1}\ns_mean_2 = {s_mean_2}")

x_mean_1 = 547.7777777777778
x_mean_2 = 547.1773485513609
s_mean_1 = 547.7777777777778
s_mean_2 = 549.2892582247113


## Kontrol Chart dengan Midrange

In [59]:
df = pl.read_excel("tabel_3_3.xlsx")
df

jam1,jam2,jam3,jam4,jam5
i64,i64,i64,i64,i64
36,35,34,33,32
31,31,34,32,30
30,30,32,30,32
32,33,33,32,35
32,34,37,37,35
…,…,…,…,…
28,31,33,33,33
33,30,34,33,35
27,28,29,27,30


In [61]:

def cc_midrange(data, A4, D5, D6):
    
    M=data.select([
        ((data.max_horizontal() + data.min_horizontal())/2).alias("M")
    ]).to_series()
    R=data.select([    
        (data.max_horizontal() - data.min_horizontal()).alias("R")
        
    ]).to_series()
    x_mean=data.select([    
        data.mean_horizontal().alias("x_mean"),
    ]).to_series()

    data = data.hstack([M,x_mean,R])
    median_M = data["M"].median()
    median_R = data["R"].median()
    LKA_M = median_M + A4*median_R
    LKB_M = median_M - A4*median_R
    LKA_R = D6*median_R
    LKB_R = D5*median_R

    print(f"LKA_M = {LKA_M}, LKB_R = {LKB_M}\nLKA_R = {LKA_R}, LKB_R = {LKB_R}")

cc_midrange(data=df, A4=0.679, D5=0, D6=2.179)



LKA_M = 36.895, LKB_R = 30.105
LKA_R = 10.895, LKB_R = 0.0
